# EDA
Whatever the data project, it always has at least one exploratory phase

In [2]:
import pandas as pd
from pandas_profiling import ProfileReport

#EDA using pandas-profiling
profile = ProfileReport(pd.read_csv('train_auto.csv'), explorative=True)

#Saving results to a HTML file
profile.to_file("output.html") 

/mnt/c/Users/Filippo/Desktop/Test ML Engineer - TIM/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_2003/597024502.py:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 66.19it/s]


Looking the report generated from the command above:
- Imbalanced dataset (very common in frauds area )
- HomeKids has high correlation with age
- YOJ has 5.6% missing values
- Income has 5.5% missing value, but most important it needs to be converted to a number format
- HOME_VAL 5.7% missing values, but most important it needs to be converted to a number format
- MSTATUS needs to be converted to a boolean
- SEX can be converted in several way, but for now will be converted to M and F
- Education will be cleaned removing z_
- Job will be cleaned removing z_,  6.4% missing values
- CAR_USE can be converted to COMMERCIAL_CAR_USE and can be a boolean
- BLUEBOOK needs to be converted to a number format
- CAR_TYPE will be cleaned removing z_
- OLDCLAIM needs to be converted to a number format
- CAR_AGE 6.2% missing values
- URBANCITY needs to be converted to a boolean
- i will use INDEX as an id


With the information above i will create the desired features store definition in the feast folder

In [12]:
import pandas as pd
df = pd.read_csv('train_auto.csv')

## define preprocess for first ETL

In [4]:
df

,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,...,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CAR_AGE,URBANICITY
0,1,0,0.0,0,60.0,0,11.0,"$67,349",No,$0,...,"$14,230",11,Minivan,yes,"$4,461",2,No,3,18.0,Highly Urban/ Urban
1,2,0,0.0,0,43.0,0,11.0,"$91,449",No,"$257,252",...,"$14,940",1,Minivan,yes,$0,0,No,0,1.0,Highly Urban/ Urban
2,4,0,0.0,0,35.0,1,10.0,"$16,039",No,"$124,191",...,"$4,010",4,z_SUV,no,"$38,690",2,No,3,10.0,Highly Urban/ Urban
3,5,0,0.0,0,51.0,0,14.0,NaN,No,"$306,251",...,"$15,440",7,Minivan,yes,$0,0,No,0,6.0,Highly Urban/ Urban
4,6,0,0.0,0,50.0,0,NaN,"$114,986",No,"$243,925",...,"$18,000",1,z_SUV,no,"$19,217",2,Yes,3,17.0,Highly Urban/ Urban
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8156,10297,0,0.0,0,35.0,0,11.0,"$43,112",No,$0,...,"$27,330",10,Panel Truck,yes,$0,0,No,0,8.0,z_Highly Rural/ Rural
8157,10298,0,0.0,1,45.0,2,9.0,"$164,669",No,"$386,273",...,"$13,270",15,Minivan,no,$0,0,No,2,17.0,Highly Urban/ Urban
8158,10299,0,0.0,0,46.0,0,9.0,"$107,204",No,"$332,591",...,"$24,490",6,Panel Truck,no,$0,0,No,0,1.0,Highly Urban/ Urban
8159,10301,0,0.0,0,50.0,0,7.0,"$43,445",No,"$149,248",...,"$22,550",6,Minivan,no,$0,0,No,0,11.0,Highly Urban/ Urban


In [95]:
import re

def nullable_dollars(value):
    if value is None or str(value) in ['NaN', 'nan']:
        return None
    if not isinstance(value, str):
        raise Exception(f'dollars should be None or string, provided value = {value}')
    if not value.startswith("$"):
        raise Exception(f'dollars should start with $, provided value = {value}')
    return float(value.replace("$", "").replace(",", ""))

def boolean(value):
    if isinstance(value, bool):
        return int(value)
    if isinstance(value, str):
        value = value.lower()
    if value in ['yes', 'y', 1]:
        return 1
    if value in ['no', 'n', 0]:
        return 0
    raise Exception(f'not a boolean, provided value = {value}')

def nullable_str(value):
    if value is None or str(value) in ['NaN', 'nan']:
        return None
    value = value.lower()
    if value.startswith("z_"):
        value = value[2:]
    return re.sub(r"[^a-zA-Z0-9]+", ' ',  value)

def sex(value):
    if not isinstance(value, str):
        raise Exception(f'sex should be a string, provided value = {value}')
    if value in ['f', 'female']:
        return 'f'
    if value in ['m', 'male']:
        return 'm'
    raise Exception(f'not a sex, provided value = {value}')

def commercial_car_use(value):
    if not isinstance(value, str):
        raise Exception(f'car use should be a string, provided value = {value}')
    if value == 'commercial':
        return 1
    if value == 'private':
        return 0
    raise Exception(f'not a car use, provided value = {value}')

def urban_city(value):
    if not isinstance(value, str):
        raise Exception(f'urban city should be a string, provided value = {value}')
    if value.startswith('highly urban'):
        return 1
    if value.startswith('highly rural'):
        return 0
    raise Exception(f'not a urban city boolean, provided value = {value}')

def education_level(value):
    order = ['<High School', 'z_High School', 'Bachelors', 'Masters', 'PhD']
    return order.index(value)

def nullable_int(value):
    if value is None or str(value) in ['NaN', 'nan']:
        return None
    return int(value)


In [96]:
commercial_car_use("private")

0

In [97]:
MAPPING_NAMES = {
    'INDEX': 'id',
    'TARGET_FLAG': 'label',
    'TARGET_AMT': 'amt',
    'KIDSDRIV': 'kids_driv',
    'AGE': 'age',
    'HOMEKIDS': 'home_kids',
    'YOJ': 'yoj',
    'INCOME': 'income',
    'PARENT1': 'parent1',
    'HOME_VAL': 'home_val',
    'MSTATUS': 'm_status',
    'SEX': 'sex',
    'EDUCATION': 'education_level',
    'JOB': 'job',
    'TRAVTIME': 'trav_time',
    'CAR_USE': 'commercial_car_use',
    'BLUEBOOK': 'blue_book',
    'TIF': 'tif',
    'CAR_TYPE': 'car_type',
    'RED_CAR': 'red_car',
    'OLDCLAIM': 'old_claim',
    'CLM_FREQ': 'clm_freq',
    'REVOKED': 'revoced',
    'MVR_PTS': 'mvr_pts',
    'CAR_AGE': 'car_age',
    'URBANICITY': 'urban_city',
}

TRANSFORM_FUNCTIONS = {
    'id': int,
    'label': boolean,
    'amt': float,
    'kids_driv': int,
    'age': nullable_int,
    'home_kids': int,
    'yoj': nullable_int,
    'income': nullable_dollars,
    'parent1': boolean,
    'home_val': nullable_dollars,
    'm_status': lambda v: boolean(nullable_str(v)),
    'sex': lambda v: sex(nullable_str(v)),
    'education_level': education_level,
    'job': nullable_str,
    'trav_time': int,
    'commercial_car_use': lambda v: commercial_car_use(nullable_str(v)),
    'blue_book': nullable_dollars,
    'tif': int,
    'car_type': nullable_str,
    'red_car': boolean,
    'old_claim': nullable_dollars,
    'clm_freq': int,
    'revoced': boolean,
    'mvr_pts': int,
    'car_age': nullable_int,
    'urban_city': lambda v: urban_city(nullable_str(v)),
}



In [92]:
df = df.rename(columns=MAPPING_NAMES)

In [98]:
df

,id,label,amt,kids_driv,age,home_kids,yoj,income,parent1,home_val,...,blue_book,tif,car_type,red_car,old_claim,clm_freq,revoced,mvr_pts,car_age,urban_city
0,1,0,0.0,0,60.0,0,11.0,"$67,349",No,$0,...,"$14,230",11,Minivan,yes,"$4,461",2,No,3,18.0,Highly Urban/ Urban
1,2,0,0.0,0,43.0,0,11.0,"$91,449",No,"$257,252",...,"$14,940",1,Minivan,yes,$0,0,No,0,1.0,Highly Urban/ Urban
2,4,0,0.0,0,35.0,1,10.0,"$16,039",No,"$124,191",...,"$4,010",4,z_SUV,no,"$38,690",2,No,3,10.0,Highly Urban/ Urban
3,5,0,0.0,0,51.0,0,14.0,NaN,No,"$306,251",...,"$15,440",7,Minivan,yes,$0,0,No,0,6.0,Highly Urban/ Urban
4,6,0,0.0,0,50.0,0,NaN,"$114,986",No,"$243,925",...,"$18,000",1,z_SUV,no,"$19,217",2,Yes,3,17.0,Highly Urban/ Urban
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8156,10297,0,0.0,0,35.0,0,11.0,"$43,112",No,$0,...,"$27,330",10,Panel Truck,yes,$0,0,No,0,8.0,z_Highly Rural/ Rural
8157,10298,0,0.0,1,45.0,2,9.0,"$164,669",No,"$386,273",...,"$13,270",15,Minivan,no,$0,0,No,2,17.0,Highly Urban/ Urban
8158,10299,0,0.0,0,46.0,0,9.0,"$107,204",No,"$332,591",...,"$24,490",6,Panel Truck,no,$0,0,No,0,1.0,Highly Urban/ Urban
8159,10301,0,0.0,0,50.0,0,7.0,"$43,445",No,"$149,248",...,"$22,550",6,Minivan,no,$0,0,No,0,11.0,Highly Urban/ Urban


In [99]:
df.transform(TRANSFORM_FUNCTIONS)

,id,label,amt,kids_driv,age,home_kids,yoj,income,parent1,home_val,...,blue_book,tif,car_type,red_car,old_claim,clm_freq,revoced,mvr_pts,car_age,urban_city
0,1,0,0.0,0,60.0,0,11.0,67349.0,0,0.0,...,14230.0,11,minivan,1,4461.0,2,0,3,18.0,1
1,2,0,0.0,0,43.0,0,11.0,91449.0,0,257252.0,...,14940.0,1,minivan,1,0.0,0,0,0,1.0,1
2,4,0,0.0,0,35.0,1,10.0,16039.0,0,124191.0,...,4010.0,4,suv,0,38690.0,2,0,3,10.0,1
3,5,0,0.0,0,51.0,0,14.0,NaN,0,306251.0,...,15440.0,7,minivan,1,0.0,0,0,0,6.0,1
4,6,0,0.0,0,50.0,0,NaN,114986.0,0,243925.0,...,18000.0,1,suv,0,19217.0,2,1,3,17.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8156,10297,0,0.0,0,35.0,0,11.0,43112.0,0,0.0,...,27330.0,10,panel truck,1,0.0,0,0,0,8.0,0
8157,10298,0,0.0,1,45.0,2,9.0,164669.0,0,386273.0,...,13270.0,15,minivan,0,0.0,0,0,2,17.0,1
8158,10299,0,0.0,0,46.0,0,9.0,107204.0,0,332591.0,...,24490.0,6,panel truck,0,0.0,0,0,0,1.0,1
8159,10301,0,0.0,0,50.0,0,7.0,43445.0,0,149248.0,...,22550.0,6,minivan,0,0.0,0,0,0,11.0,1


In [101]:
from pandas_profiling import ProfileReport
#EDA using pandas-profiling
profile = ProfileReport(df.transform(TRANSFORM_FUNCTIONS), explorative=True)
#Saving results to a HTML file
profile.to_file("after_etl.html") 

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 10.34it/s]
